In [68]:
import os
import time
from openai import OpenAI

# --- Set up client ---
os.environ["GROQ_API_KEY"] = "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"
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)


In [70]:
# --- Conversation storage ---
conversation_history = []

def add_message(role, content):
    conversation_history.append({"role": role, "content": content})

def get_groq_response():
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=conversation_history
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error getting response: {e}")
        return "I apologize, but I'm having trouble responding at the moment."

In [71]:
# --- Simplified Summarization function ---
def summarize_history():
    """Summarize the conversation and add it as a system message"""
    global conversation_history

    # Remove any existing summary
    if conversation_history and conversation_history[0]["role"] == "system" and "[Summary]" in conversation_history[0]["content"]:
        conversation_history.pop(0)

    # Get the last 6 messages (3 user + 3 assistant)
    if len(conversation_history) < 4:
        print("Not enough messages to summarize")
        return

    messages_to_summarize = conversation_history[-6:]  # Last 3 exchanges

    try:
        # Create a simple prompt for summarization
        prompt = "Please summarize the following conversation in 1-2 sentences:\n\n"
        for msg in messages_to_summarize:
            role = "User" if msg["role"] == "user" else "Assistant"
            prompt += f"{role}: {msg['content']}\n"

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes conversations concisely."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.1
        )
        summary_text = response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error during summarization: {e}")
        # Create a simple fallback summary
        user_messages = [msg["content"] for msg in messages_to_summarize if msg["role"] == "user"]
        summary_text = f"Conversation about: {', '.join(user_messages[:2])}"

    # Insert summary at the top
    conversation_history.insert(0, {"role": "system", "content": f"[Summary]: {summary_text}"})
    return summary_text


In [73]:
# --- Pretty print history ---
def print_history():
    print("\n--- Conversation History ---")
    for idx, msg in enumerate(conversation_history, 1):
        role = msg["role"].upper()
        content = msg['content']
        if len(content) > 100:
            content = content[:97] + "..."
        print(f"{idx}. {role}: {content}")
    print("---------------------------\n")

# --- Continuous chat ---
conversation_run_count = 0
K = 3  # summarize every 3 chats

def start_chat():
    global conversation_run_count

    print("Type 'exit' or 'quit' to end the chat.")
    print("Type 'history' to view current conversation history.")
    print("Type 'summary' to manually summarize the conversation.\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("\n[Chat ended]")
            break
        elif user_input.lower() == "history":
            print_history()
            continue
        elif user_input.lower() == "summary":
            summary = summarize_history()
            print(f"\nSummary: {summary}")
            print_history()
            continue

        add_message("user", user_input)
        assistant_reply = get_groq_response()
        add_message("assistant", assistant_reply)

        conversation_run_count += 1

        # Keep only the last 10 messages
        if len(conversation_history) > 10:
            conversation_history.pop(0)  # Remove the oldest message

        # Summarize after every K chats
        if conversation_run_count % K == 0:
            print(f"\n[Summarizing conversation after {K} exchanges]")
            summary = summarize_history()
            print(f"Summary: {summary}")
            print_history()

        print(f"Assistant: {assistant_reply}")

# --- Main execution ---



In [74]:
if __name__ == "__main__":
    print("Conversation Management System with Summarization")
    print("================================================")
start_chat()

Conversation Management System with Summarization
Type 'exit' or 'quit' to end the chat.
Type 'history' to view current conversation history.
Type 'summary' to manually summarize the conversation.

You: hi good morning
Assistant: Good morning! Hope you're having a great start to the day! Is there something I can help you with or would you like to chat?
You: i just woke up
Assistant: Welcome to the land of the awake! It can take a little while to shake off the sleepiness, but I'm here to help get you started. How about a gentle morning question: Did you have any interesting dreams or are you looking forward to something specific today?
You: i want to go college

[Summarizing conversation after 3 exchanges]
Summary: The user woke up and is having a gentle morning conversation, sharing their goal of attending college. The assistant is offering support and guidance, inquiring about the user's interests and plans for college, such as potential majors or career paths, and whether they've sta

In [75]:
import os
import json
from openai import OpenAI

# --- Set up client ---
os.environ["GROQ_API_KEY"] = "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"
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

# --- JSON Schema for Information Extraction ---
user_info_schema = {
    "name": "extract_user_information",
    "description": "Extract user information from the conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The user's full name"
            },
            "email": {
                "type": "string",
                "description": "The user's email address"
            },
            "phone": {
                "type": "string",
                "description": "The user's phone number"
            },
            "location": {
                "type": "string",
                "description": "The user's location or address"
            },
            "age": {
                "type": "integer",
                "description": "The user's age"
            }
        },
        "required": []  # All fields are optional
    }
}

# --- Sample conversations for demonstration ---
sample_conversations = [
    [
        {"role": "user", "content": "Hi, my name is John Doe and I'm 25 years old."},
        {"role": "assistant", "content": "Nice to meet you, John! How can I help you today?"},
        {"role": "user", "content": "I live in New York and my email is johndoe@example.com."}
    ],
    [
        {"role": "user", "content": "You can reach me at 555-123-4567."},
        {"role": "assistant", "content": "Thank you for providing your phone number. Is there anything else you'd like to share?"},
        {"role": "user", "content": "My name is Alice Smith and I'm from London."}
    ],
    [
        {"role": "user", "content": "I'm 30 years old and my email is alice.smith@company.com."},
        {"role": "assistant", "content": "Got it! Is there anything specific you need help with?"},
        {"role": "user", "content": "Yes, I need help with booking a flight to Paris."}
    ]
]

def extract_information(conversation):
    """Extract structured information from a conversation using function calling"""
    extracted_info = {}

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=conversation,
            functions=[user_info_schema],
            function_call={"name": "extract_user_information"}
        )

        # Check if the model wants to call a function
        if response.choices[0].message.function_call:
            function_call = response.choices[0].message.function_call
            if function_call.name == "extract_user_information":
                # Parse the arguments
                arguments = json.loads(function_call.arguments)

                # Update extracted info with new values
                for key, value in arguments.items():
                    if value:  # Only include if value is not empty
                        extracted_info[key] = value

                return extracted_info
        return {}

    except Exception as e:
        print(f"Error during information extraction: {e}")
        return {}

def validate_extracted_info(extracted_info):
    """Validate extracted information against the schema"""
    schema_properties = user_info_schema["parameters"]["properties"]
    validation_results = {"valid": True, "details": []}

    for field, value in extracted_info.items():
        if field in schema_properties:
            expected_type = schema_properties[field]["type"]
            actual_type = type(value).__name__

            # Check type compatibility
            if (expected_type == "string" and isinstance(value, str)) or \
               (expected_type == "integer" and isinstance(value, int)):
                validation_results["details"].append(
                    f"✓ {field}: Type matches schema ({expected_type})"
                )
            else:
                validation_results["details"].append(
                    f"✗ {field}: Type mismatch (expected {expected_type}, got {actual_type})"
                )
                validation_results["valid"] = False
        else:
            validation_results["details"].append(
                f"⚠ {field}: Not defined in schema"
            )

    # Check for missing required fields (if any were required)
    required_fields = user_info_schema["parameters"]["required"]
    for field in required_fields:
        if field not in extracted_info:
            validation_results["details"].append(
                f"⚠ {field}: Missing required field"
            )
            validation_results["valid"] = False

    return validation_results

def demonstrate_information_extraction():
    """Demonstrate information extraction from sample conversations"""
    print("=== TASK 2: JSON Schema Classification & Information Extraction ===")
    print("Schema defines extraction of: name, email, phone, location, age\n")

    for i, conversation in enumerate(sample_conversations, 1):
        print(f"\n--- Sample Conversation {i} ---")

        # Display the conversation
        for msg in conversation:
            print(f"{msg['role'].capitalize()}: {msg['content']}")

        # Extract information
        extracted_info = extract_information(conversation)

        print("\n--- Extracted Information ---")
        if extracted_info:
            for key, value in extracted_info.items():
                print(f"{key.capitalize()}: {value}")
        else:
            print("No information extracted.")

        # Validate against schema
        print("\n--- Validation Against Schema ---")
        if extracted_info:
            validation = validate_extracted_info(extracted_info)
            for detail in validation["details"]:
                print(detail)
            print(f"\nOverall validation: {'PASS' if validation['valid'] else 'FAIL'}")
        else:
            print("No information to validate.")

        print("\n" + "="*60)

# --- Main execution ---
if __name__ == "__main__":
    demonstrate_information_extraction()

=== TASK 2: JSON Schema Classification & Information Extraction ===
Schema defines extraction of: name, email, phone, location, age


--- Sample Conversation 1 ---
User: Hi, my name is John Doe and I'm 25 years old.
Assistant: Nice to meet you, John! How can I help you today?
User: I live in New York and my email is johndoe@example.com.

--- Extracted Information ---
Age: 25
Email: johndoe@example.com
Location: New York
Name: John Doe

--- Validation Against Schema ---
✓ age: Type matches schema (integer)
✓ email: Type matches schema (string)
✓ location: Type matches schema (string)
✓ name: Type matches schema (string)

Overall validation: PASS


--- Sample Conversation 2 ---
User: You can reach me at 555-123-4567.
Assistant: Thank you for providing your phone number. Is there anything else you'd like to share?
User: My name is Alice Smith and I'm from London.

--- Extracted Information ---
Location: London
Name: Alice Smith
Phone: 555-123-4567

--- Validation Against Schema ---
✓ loca